In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import joblib
import keras

In [ ]:
data = pd.read_csv("synthetic_patient_data2.csv")
data.head()

,fever,cough,fatigue,age,gender,Difficulty Breathing,Blood Pressure,Cholesterol Level,diagnosis,treatment_plan
0,101.115414,0,0,69,Male,1,98.612450,177.117130,Urinary Tract Infection,Vitamin C intake
1,96.200086,1,1,72,Male,1,92.497900,215.476284,Eczema,Inhalers and tablets
2,98.200235,1,1,20,Female,1,105.366867,145.523232,Hepatitis,Vitamin C intake
3,97.785686,0,1,36,Female,0,129.684855,141.031570,Hyperthyroidism,Inhalers and tablets
4,101.891770,1,1,19,Male,1,112.809230,127.949189,Urinary Tract Infection,"surgery, chemotherapy, using heat to destroy t..."


In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.model_selection import train_test_split

X = data[['fever', 'cough', 'fatigue', 'age', 'Difficulty Breathing', 'Blood Pressure', 'Cholesterol Level']]

label_encoder_diagnosis = LabelEncoder()
y_diagnosis = label_encoder_diagnosis.fit_transform(data['diagnosis'])
joblib.dump(label_encoder_diagnosis, 'label_encoder_diagnosis.pkl')

label_encoder_treatment = LabelEncoder()
y_treatment = label_encoder_treatment.fit_transform(data['treatment_plan'])
joblib.dump(label_encoder_treatment, 'label_encoder_treatment.pkl')

X_train, X_test, y_diagnosis_train, y_diagnosis_test, y_treatment_train, y_treatment_test = train_test_split(
    X, y_diagnosis, y_treatment, test_size=0.2, random_state=42
)
scaler = StandardScaler()
scaler.fit(X_train)

# Save the scaler to a file
joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
num_classes = 8
# Convert target labels to integer encoding
label_encoder_diagnosis = LabelEncoder()
y_diagnosis_train_encoded = label_encoder_diagnosis.fit_transform(y_diagnosis_train)

# Assuming you have 8 classes for treatment plans
num_treatment_classes = 8

# Convert target labels to integer encoding for treatment plans
label_encoder_treatment = LabelEncoder()
y_treatment_train_encoded = label_encoder_treatment.fit_transform(y_treatment_train)



In [ ]:

# Define the neural network model with two outputs
input_layer = Input(shape=(X_train_scaled.shape[1],))
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)

#Output layers for the two predictions
diagnosis_output = Dense(8, activation='softmax', name='diagnosis_output')(x)
treatment_output = Dense(8, activation='softmax', name='treatment_output')(x)

# The actual model creation
model = Model(inputs=input_layer, outputs=[diagnosis_output, treatment_output])


In [ ]:
pip

In [ ]:
#Compile the model with multiple loss functions, one for each output
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'diagnosis_output': 'sparse_categorical_crossentropy',
                    'treatment_output': 'sparse_categorical_crossentropy'},
              metrics=['accuracy'])
#Train the model with the respective data for each output
model.fit(X_train_scaled,
          {'diagnosis_output': y_diagnosis_train, 'treatment_output': y_treatment_train},
          epochs=30000,
          batch_size=32,
          validation_split=0.2)


Streaming output truncated to the last 5000 lines.
20/20 [==============================] - 0s 8ms/step - loss: 0.0228 - diagnosis_output_loss: 0.0117 - treatment_output_loss: 0.0111 - diagnosis_output_accuracy: 0.9937 - treatment_output_accuracy: 0.9953 - val_loss: 170.7611 - val_diagnosis_output_loss: 90.8184 - val_treatment_output_loss: 79.9427 - val_diagnosis_output_accuracy: 0.0875 - val_treatment_output_accuracy: 0.1375
Epoch 27502/30000
20/20 [==============================] - 0s 8ms/step - loss: 0.0226 - diagnosis_output_loss: 0.0115 - treatment_output_loss: 0.0111 - diagnosis_output_accuracy: 0.9953 - treatment_output_accuracy: 0.9953 - val_loss: 170.5634 - val_diagnosis_output_loss: 90.6831 - val_treatment_output_loss: 79.8803 - val_diagnosis_output_accuracy: 0.0875 - val_treatment_output_accuracy: 0.1375
Epoch 27503/30000
20/20 [==============================] - 0s 9ms/step - loss: 0.0226 - diagnosis_output_loss: 0.0115 - treatment_output_loss: 0.0111 - diagnosis_output_accu

In [ ]:
model.save('folmy_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 7)]                  0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   512       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 16)                   528       ['dense_1[0][0]']             
                                                                                              

In [ ]:
# Evaluate the model
diagnosis_accuracy = model.evaluate(X_test_scaled, y_diagnosis_test)[1]
treatment_accuracy = model.evaluate(X_test_scaled, y_treatment_test)[1]

print(f"Diagnosis Accuracy: {diagnosis_accuracy}, Treatment Accuracy: {treatment_accuracy}")

7/7 [==============================] - 0s 5ms/step - loss: 166.7540 - diagnosis_output_loss: 82.3126 - treatment_output_loss: 84.4414 - diagnosis_output_accuracy: 0.1200 - treatment_output_accuracy: 0.1500
Diagnosis Accuracy: 91.66862487792969, Treatment Accuracy: 82.31263732910156


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
def get_user_input():
    print("Enter new patient details:")
    fever = float(input("Fever: "))
    cough = int(input("Cough (0 for No, 1 for Yes): "))
    fatigue = int(input("Fatigue (0 for No, 1 for Yes): "))
    age = int(input("Age: "))
    difficulty_breathing = int(input("Difficulty Breathing (0 for No, 1 for Yes): "))
    blood_pressure = float(input("Blood Pressure: "))
    cholesterol_level = float(input("Cholesterol Level: "))
    ## Assuming you have a NumPy array with the same structure as X_train_scaled
    # p = [101.11541438766307,0,0,69,1,98.61245044792497,177.11712962274254]
    new_patient_data = np.array([[fever, cough, fatigue, age, difficulty_breathing, blood_pressure, cholesterol_level]])
    # new_patient_data = np.array([p])
    return new_patient_data

In [ ]:
def preprocess_new_patient_data(new_patient_data, scaler):
    standardized_data = scaler.transform(new_patient_data)
    return standardized_data

In [ ]:
def predict_diagnosis_treatment(model, new_patient_data, label_encoder_diagnosis, label_encoder_treatment):
    # Preprocess the new patient data
    preprocessed_data = preprocess_new_patient_data(new_patient_data, scaler)

    # Make predictions
    predictions = model.predict(preprocessed_data)

    diagnosis_predictions = predictions[0]
    treatment_predictions = predictions[1]
    print(diagnosis_predictions)
    print(treatment_predictions)
    predicted_diagnosis_class = np.argmax(diagnosis_predictions, axis=1)
    predicted_treatment_class = np.argmax(treatment_predictions, axis=1)

    # Decode predictions
    decoded_predictions_diagnosis = label_encoder_diagnosis.inverse_transform([predicted_diagnosis_class])[0]
    decoded_predictions_treatment = label_encoder_treatment.inverse_transform([predicted_treatment_class])[0]

    return decoded_predictions_diagnosis, decoded_predictions_treatment

In [ ]:
model = keras.models.load_model('my_model.h5')

label_encoder_diagnosis = joblib.load('label_encoder_diagnosis.pkl')
label_encoder_treatment = joblib.load('label_encoder_treatment.pkl')

In [ ]:
new_patient_data = get_user_input()
print(new_patient_data)

Enter new patient details:
Fever: 100
Cough (0 for No, 1 for Yes): 1
Fatigue (0 for No, 1 for Yes): 0
Age: 28
Difficulty Breathing (0 for No, 1 for Yes): 0
Blood Pressure: 111
Cholesterol Level: 11
[[100.   1.   0.  28.   0. 111.  11.]]


In [ ]:
#Make predictions
predicted_diagnosis, predicted_treatment = predict_diagnosis_treatment(model, new_patient_data, label_encoder_diagnosis, label_encoder_treatment)

print(f"predicted_diagnosis: {predicted_diagnosis} predicted_treatment:{predicted_treatment}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 165ms/step
[[2.1918247e-02 1.3605476e-26 7.3924018e-12 3.9509271e-07 9.7808141e-01
  1.0156137e-14 1.7672874e-26 1.5015703e-13]]
[[0.0000000e+00 8.5037925e-19 4.6090789e-27 9.9999988e-01 0.0000000e+00
  2.2741866e-27 0.0000000e+00 1.4430137e-07]]
predicted_diagnosis: Hepatitis predicted_treatment:Vitamin C intake


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
X_test.head()

,fever,cough,fatigue,age,Difficulty Breathing,Blood Pressure,Cholesterol Level
521,98.799683,1,0,60,1,132.293076,252.464303
737,96.729939,0,0,61,0,110.115048,170.027945
740,101.361068,0,0,37,1,128.172111,120.928671
660,100.502717,1,0,55,1,115.334252,257.243743
411,102.009105,0,1,32,0,134.634326,139.879543


In [ ]:
from keras.models import load_model

# Assuming 'model' is your Keras model
model.save('savemodel.h5')
loaded_model = load_model('savemodel.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
